### Libraries and Definitons

In [27]:
import os
import time
import pynq

def set_vio(vio_1, vio_2):
    #Voltage divider before VFB1, VFB2
    R1 = 22   #R119, R125 
    R2 = 24.9 #R120, R126
    
    vio_min = 0.6 * (R1 + R2) / R2   
    vio_max = 1.87 * (R1 + R2) / R2
    
    if vio_1 != 0 and (vio_1 < vio_min or vio_1 > vio_max):
        print(f"VIO 1 out of range. Accepted values 0 and [{vio_min:.2f}, {vio_max:.2f}].")
        return
    
    if vio_2 != 0 and (vio_2 < vio_min or vio_2 > vio_max):
        print(f"VIO 2 out of range. Accepted values 0 and [{vio_min:.2f}, {vio_max:.2f}].")
        return  
    
    vio_1_en = vio_1 != 0
    vio_2_en = vio_2 != 0
    vio_1_mio = 0
    vio_2_mio = 7 
    
    if vio_1_en:
        vref = int((vio_1 * R2 / (R1 + R2) - 0.6) / 0.01)
        os.system("i2cset -y 0 0x6a 0x0 0x0")      #TPS65400 set PAGE to 0 
        os.system("i2cset -y 0 0x6a 0x10 0x20")    #TPS65400 set WRITE_PROTECT to 0x20 
        os.system(f"i2cset -y 0 0x6a 0xd8 {vref}") #TPS65400 set VREF
        
    if vio_2_en:
        vref = int((vio_2 * R2 / (R1 + R2) - 0.6) / 0.01)
        os.system("i2cset -y 0 0x6a 0x0 0x1")      #TPS65400 set PAGE to 1 
        os.system("i2cset -y 0 0x6a 0x10 0x20")    #TPS65400 set WRITE_PROTECT to 0x20 
        os.system(f"i2cset -y 0 0x6a 0xd8 {vref}") #TPS65400 set VREF

    #GPIO_RST_CTRL (enable GPIO clk logic reset)
    reg = pynq.mmio.MMIO(base_addr=0xF800022C, length=4)
    reg.write(0x0, 1)

    #APER_CLK_CTRL (enable GPIO AMBA Clock)
    reg = pynq.mmio.MMIO(base_addr=0xF800012C, length=4)
    mask = ~(1 << 22)
    data = (reg.read() & mask) | (1 << 22)
    reg.write(0x0, data)

    #GPIO_RST_CTRL (disable GPIO clk logic reset)
    reg = pynq.mmio.MMIO(base_addr=0xF800022C, length=4)
    reg.write(0x0, 0)

    #MIO_PIN_00 (use GPIO and CMOS_33)
    addr = 0xF8000700
    reg = pynq.mmio.MMIO(base_addr=addr, length=4)
    data = 0x3 << 9
    reg.write(0x0, data)

    #MIO_PIN_07 (use GPIO and CMOS_33)
    addr=0xF800071C
    reg = pynq.mmio.MMIO(base_addr=addr, length=4)
    data = 0x3 << 9
    reg.write(0x0, data)

    #XGPIOPS_DIRM_OFFSET (GPIO direction)
    addr = 0xE000A204
    reg = pynq.mmio.MMIO(base_addr=addr, length=4)
    mask = ~((1 << vio_1_mio ) | (1 << vio_2_mio))
    data = (reg.read() & mask) | (1 << vio_1_mio ) | (1 << vio_2_mio)
    reg.write(0x0, data)

    #XGPIOPS_OUTEN_OFFSET (GPIO output enbale)
    addr = 0xE000A208
    reg = pynq.mmio.MMIO(base_addr=addr, length=4)
    mask = ~((1 << vio_1_mio ) | (1 << vio_2_mio))
    data = (reg.read() & mask) | (1 << vio_1_mio ) | (1 << vio_2_mio)
    reg.write(0x0, data)

    #XGPIOPS_DATA_LSW_OFFSET (GPIO value)
    addr = 0xE000A000
    reg = pynq.mmio.MMIO(base_addr=addr, length=4)
    mask = 0xffff ^ ((1 << vio_1_mio ) | (1 << vio_2_mio))
    data = (mask << 16) | (vio_1_en << vio_1_mio ) | (vio_2_en << vio_2_mio)
    reg.write(0x0, data)


### Set VIO voltages

In [54]:
set_vio(vio_1=3.3, vio_2=3.3)

### Load Overlay

In [ ]:
ol = pynq.Overlay("base.bit")

### Blink LEDs

In [21]:
for i in range(8):
    ol.axi_gpio_0.channel1.write(val=(1<<i), mask=0xff)
    time.sleep(0.5)